<a href="https://colab.research.google.com/github/noraleilaa/460_FINAL/blob/main/GasStation_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
#!pip install simpy

import pandas as pd
import simpy
import numpy as np
import random



def customer(env, name, counter):
    start_time = env.now
    tank_percentage = random.uniform(0, 50)  # Random tank percentage for each customer
    activity_time = (100-tank_percentage) / 10  # Assuming 10% full tank corresponds to 1 unit of activity time as per national constraints
    print(f'{name}: Arrived at time {start_time}, Tank Percentage: {tank_percentage}%')

    with counter.request() as req:
        yield req
        print(f'{name}: Seized gas pump at time {env.now}')
        yield env.timeout(activity_time)
        print(f'{name}: Released from gas pump at time {env.now}')

    end_time = env.now
    print(f'{name}: Finished at time {end_time}')

    return {'start_time': start_time, 'end_time': end_time, 'activity_time': activity_time}

def customer_generator(env, counter, total_customers):
    customer_id = 1
    while customer_id <= total_customers:
        yield env.timeout(np.random.exponential(5))
        env.process(customer(env, f'Customer {customer_id}', counter))
        customer_id += 1

def shop_simulation(env, total_customers):
    counter = simpy.Resource(env, capacity=4)  # change capacity of gas station pumps

    # simulation results
    results = []

    def customer_wrapper():
        nonlocal results
        while True:
            result = yield from customer(env, f'Customer {len(results) + 1}', counter)
            results.append(result)

    env.process(customer_generator(env, counter, total_customers))
    env.process(customer_wrapper())

    # duration of results
    env.run(until=1000)

    return results

env = simpy.Environment()

shop_data = shop_simulation(env, total_customers=100)
shop_data_df = pd.DataFrame(shop_data)

# Calculate waiting time
shop_data_df['waiting_time'] = shop_data_df['end_time'] - shop_data_df['start_time'] - shop_data_df['activity_time']
waiting_time = shop_data_df['waiting_time']

print(shop_data_df)


Customer 1: Arrived at time 0, Tank Percentage: 36.51571561876184%
Customer 1: Seized gas pump at time 0
Customer 1: Arrived at time 3.2481875661901527, Tank Percentage: 24.947930771779237%
Customer 1: Seized gas pump at time 3.2481875661901527
Customer 2: Arrived at time 5.655794209290244, Tank Percentage: 28.49790175976411%
Customer 2: Seized gas pump at time 5.655794209290244
Customer 1: Released from gas pump at time 6.3484284381238165
Customer 1: Finished at time 6.3484284381238165
Customer 2: Arrived at time 6.3484284381238165, Tank Percentage: 28.52733970900167%
Customer 2: Seized gas pump at time 6.3484284381238165
Customer 3: Arrived at time 10.151594909270532, Tank Percentage: 23.703608816174597%
Customer 3: Seized gas pump at time 10.151594909270532
Customer 1: Released from gas pump at time 10.753394489012228
Customer 1: Finished at time 10.753394489012228
Customer 4: Arrived at time 10.919849642533446, Tank Percentage: 33.13094454737717%
Customer 4: Seized gas pump at time

In [43]:
import random
shop_data_df.loc[shop_data_df['waiting_time'] < 0, 'waiting_time'] = 0
shop_data_df['Balking'] = np.where(shop_data_df['waiting_time'] < random.uniform(0, 10), 0, 1)
shop_data_df['Amount of gas in litres'] = shop_data_df['activity_time']* 10

print(shop_data_df)

shop_data_df.to_csv('GS_data_df_4.csv')

     start_time    end_time  activity_time  waiting_time  Balking  \
0      0.000000    6.348428       6.348428  0.000000e+00        0   
1      6.348428   13.495694       7.147266  0.000000e+00        0   
2     13.495694   19.253375       5.757680  0.000000e+00        0   
3     19.253375   27.191776       7.938401  0.000000e+00        0   
4     27.191776   36.563723       9.371947  0.000000e+00        0   
..          ...         ...            ...           ...      ...   
121  953.265228  960.850053       7.584825  0.000000e+00        0   
122  960.850053  970.832993       9.982940  5.684342e-14        0   
123  970.832993  979.615497       8.782504  4.973799e-14        0   
124  979.615497  989.186715       9.571219  0.000000e+00        0   
125  989.186715  999.036900       9.850185  0.000000e+00        0   

     Amount of gas in litres  
0                  63.484284  
1                  71.472660  
2                  57.576801  
3                  79.384011  
4               

In [44]:
shop_data_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   start_time               126 non-null    float64
 1   end_time                 126 non-null    float64
 2   activity_time            126 non-null    float64
 3   waiting_time             126 non-null    float64
 4   Balking                  126 non-null    int64  
 5   Amount of gas in litres  126 non-null    float64
dtypes: float64(5), int64(1)
memory usage: 6.0 KB


In [45]:
shop_data_df.head()

,start_time,end_time,activity_time,waiting_time,Balking,Amount of gas in litres
0,0.000000,6.348428,6.348428,0.0,0,63.484284
1,6.348428,13.495694,7.147266,0.0,0,71.472660
2,13.495694,19.253375,5.757680,0.0,0,57.576801
3,19.253375,27.191776,7.938401,0.0,0,79.384011
4,27.191776,36.563723,9.371947,0.0,0,93.719473


In [46]:
##calculatin the total amount of gas sold and how that translates to money
## based on the average current rate on 11/30/2023
## the reason gas prices are so low is too much supply less use during winter months
conversion_factor = 0.95  # based on the average current rate on 11/30/2023 in th US
profit_df = pd.DataFrame({
    'gas_sold': shop_data_df['Amount of gas in litres'],
    '$_amount_of_gas': shop_data_df['Amount of gas in litres'] * conversion_factor
})

print(profit_df)



      gas_sold  $_amount_of_gas
0    63.484284        60.310070
1    71.472660        67.899027
2    57.576801        54.697961
3    79.384011        75.414810
4    93.719473        89.033500
..         ...              ...
121  75.848253        72.055840
122  99.829396        94.837926
123  87.825039        83.433787
124  95.712187        90.926577
125  98.501846        93.576754

[126 rows x 2 columns]


In [47]:
wholesale_price_of_gas = .7 ## this is an estimate

profit_df['profit'] = profit_df['gas_sold'] * (conversion_factor- wholesale_price_of_gas)

profit_df

,gas_sold,$_amount_of_gas,profit
0,63.484284,60.310070,15.871071
1,71.472660,67.899027,17.868165
2,57.576801,54.697961,14.394200
3,79.384011,75.414810,19.846003
4,93.719473,89.033500,23.429868
...,...,...,...
121,75.848253,72.055840,18.962063
122,99.829396,94.837926,24.957349
123,87.825039,83.433787,21.956260
124,95.712187,90.926577,23.928047


In [48]:
profit = sum(profit_df['profit'])
profit
profit_df.to_csv('Profit_for_4_pumps.csv')
print('The amount of profit per day:', profit)

The amount of profit per day: 2456.9799016214433
